#### Learn about Trainer() in Keras Higher Level API (vs pytorch train loop)

In [5]:
import os
os.environ['http_proxy'] = "http://127.0.0.1:7890"
os.environ['https_proxy'] = "http://127.0.0.1:7890"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from datasets import load_dataset
# tensorflow 代码常用os环境变量

In [ ]:
dataset = load_dataset("glue", 'cola')
dataset = dataset["train"]  # Just take the training split for now
dataset[0]

In [ ]:
dataset = load_dataset("yelp_review_full")
dataset = dataset["train"]
dataset[0]

In [ ]:
dataset = dataset.select(range(10000))
The screen -X command is used to send commands to a running screen session. It allows you to interact with the session without attaching to it.dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
tokenized_data = tokenizer(dataset["text"], return_tensors="np", padding="max_length")
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)

labels = np.array(dataset["label"])

In [ ]:
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["sentence"])


dataset = dataset.map(tokenize_dataset)

In [ ]:
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["text"], padding="max_length", truncation=True)


dataset = dataset.map(tokenize_dataset)

In [ ]:
dataset[0]['input_ids'].__len__()

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")

tf_dataset = model.prepare_tf_dataset(dataset, batch_size=6, shuffle=True, tokenizer=tokenizer)

In [ ]:
model.compile()  # No loss argument!

model.fit(tf_dataset)

  4671/108333 [>.............................] - ETA: 4:37:49 - loss: nan